**PARTIE 2 ANALYSE DONNEES :**

- ~~FILTRE SPAM : enlever ceux qui sont pas en fr~~
- ~~co-occurrences (bigrammes/trigrammes de mots ? hors mots vides ?)~~
- ~~regarder tokens les + fqts (hors mots vides)~~
- ~~entités les + fqtes ~~
- noms de sentiments les + fqts
- polarité la + représentée
- essayer aspect diachronique : évolution hashtags (graphiques ?)
- ~~nombre d'auteurs différents (avec author_id, pour voir si gens ont tendance à faire un ou pls tweets dessus) ~~
- ~~langue (que fr détecté ?)~~
- ~~métriques des tweets les + partagés/moyenne~~
- ~~stats sur corpus ((lieux des tweets/users, popularité des utilisateurs) ~~nb de tweets contenant un média et quel type)~~

In [1]:
import json,glob
from itertools import tee, islice
from collections import Counter
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

def ouvrir_json(chemin):
    with open(chemin, encoding="utf-8") as f:
        toto = json.load(f)
    return toto

def trier_dic(dic):
    L = [[effectif,car] for car,effectif in dic.items()]
    L_sorted = sorted(L, reverse=True)
    return [[car,effectif] for effectif,car in L_sorted]


def ngrams(lst, n): #fonction trouvée sur https://stackoverflow.com/questions/12488722/counting-bigrams-pair-of-two-words-in-a-file-using-python
    tlst = lst
    while True:
        a, b = tee(tlst)
        l = tuple(islice(a, n))
        if len(l) == n:
            yield l
            next(b)
            tlst = b
        else:
            break

/home/fanny/anaconda3/envs/main/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
#variables de base du corpus

nb_media = 0
types_media = {}

followers=[]
loc_users={}

lg = {}
tokens = {}
mentions = {}
hashtags = {}
authors = {}
dates = {}
total_text = ""
total_text_sw = ""

nb_url=0
rt, rep, like = [], [], []

for doc in glob.glob("corpus_projet_plus/*"):
    corpus = ouvrir_json(doc)
    #dico structure : 3 grandes clés : data, includes (media), meta
    data = corpus["data"]
    includes = corpus["includes"]
    meta = corpus["meta"]
    
    """PARTIE INCLUDES/MEDIAS"""
    #nombre de tweets contenant un média
    nb_media += len(includes["media"])
    for media in includes["media"]:
        types_media[media["type"]] = types_media.get(media["type"], 0) + 1
        
    """PARTIE INCLUDES/USERS"""
    #nb d'id uniques, moy et max de followers, location
    for user in includes["users"]:
        followers.append(user["public_metrics"]["followers_count"])
        try:
            loc_users[user["location"]] = loc_users.get(user["location"], 0) + 1 #pas trop utile car pas normalisé
        except KeyError:
            pass
        
    """PARTIE DATA"""

    for tweet in data:
        """TEXT"""
        text = tweet["text"].lower()
        total_text += tweet["text"].lower()

        #tokens les + fréquents (hors mots vides)
        for token in text.split():
        #for token in word_tokenize(text.lower(), language='french'): #peut-être pas une bonne idée > pb pour les mentions, liens et hashtags
            if token not in fr_stop and token.isalpha():
                tokens[token] = tokens.get(token, 0) + 1
                total_text_sw += " "+token        

        """ENTITIES : mentions (nb+dico username), urls (nb), hashtags (nb+ dico tags)"""
        try: #pour éviter pb des tweets sans entités détectées
            for mention in tweet["entities"]["mentions"]:
                mentions[mention["username"]] = mentions.get(mention["username"], 0) + 1

            nb_url += len(tweet["entities"]["urls"]) 

            for ht in tweet["entities"]["hashtags"]:
                hashtags[ht["tag"].lower()] = hashtags.get(ht["tag"].lower(), 0) + 1

        except KeyError:
            pass


        """LANG"""
        lg[tweet['lang']] = lg.get(tweet['lang'], 0) + 1
        if tweet['lang']!='fr':
            print(text)
            print("********")

        """AUTHOR_ID"""
        authors[tweet["author_id"]] = authors.get(tweet["author_id"],0)+1

        """CREATED_AT : pas les heures, donc slmt 10 premiers car de la date"""
        dates[tweet["created_at"][:10]] = dates.get(tweet["created_at"][:10], 0)+1

        """PUBLIC_METRICS : retweet_count, reply_count, like_count, (quote_count, impression_count) => moy et max"""
        rt.append(tweet["public_metrics"]["retweet_count"])
        rep.append(tweet["public_metrics"]["reply_count"])
        like.append(tweet["public_metrics"]["like_count"])

        
print("Types média", types_media)
print("\nLangues : ",lg)
print("\nDates : ", dates)
print("\nAuteurs uniques", len(authors), "et les plus présents", trier_dic(authors)[:5])
print("\nNombre moyen de followers", round(sum(followers)/len(followers),2), " nombre max :", sorted(followers)[-5:])
print("\nMax de RT : ", max(rt), "/ max de rép :", max(rep), "/ max de likes : ", max(like))
print("\nMoyennes de RT : ", sum(rt)/len(rt), "/ de rép :", sum(rep)/len(rep), "/ de likes : ", sum(like)/len(like))
print("\nHashtags : ",trier_dic(hashtags)[:50])
print("\nMentions : ",trier_dic(mentions)[:50])
print("\nTokens : ",trier_dic(tokens)[:50])

bigr = Counter(ngrams(total_text.split(), 2))
trigr = Counter(ngrams(total_text.split(), 3))
print("\n\nBigrammes : ", bigr.most_common(20))
print("\nTrigrammes : ", trigr.most_common(20))

bigr_sw = Counter(ngrams(total_text_sw.split(), 2))
trigr_sw = Counter(ngrams(total_text_sw.split(), 3))
print("\n\nBigrammes sans mots grammaticaux : ", bigr_sw.most_common(20))
print("\nTrigrammes sans mots grammaticaux : ", trigr_sw.most_common(20))

Types média {'photo': 2278, 'video': 430, 'animated_gif': 99}

Langues :  {'fr': 8109}

Dates :  {'2023-02-04': 542, '2023-02-01': 400, '2023-01-18': 470, '2023-01-21': 200, '2023-01-29': 300, '2023-02-02': 400, '2023-02-05': 610, '2023-01-26': 300, '2023-02-06': 547, '2023-02-07': 199, '2023-01-23': 400, '2023-01-22': 200, '2023-02-03': 500, '2023-01-28': 400, '2023-01-25': 300, '2023-01-17': 193, '2023-01-19': 455, '2023-01-27': 200, '2023-01-20': 300, '2023-01-30': 299, '2023-01-31': 499, '2023-01-24': 300, '2023-01-16': 95}

Auteurs uniques 5333 et les plus présents [['1564885347120603138', 175], ['306751565', 41], ['1218447306061754369', 34], ['85362553', 33], ['2966670748', 30]]

Nombre moyen de followers 438438.79  nombre max : [78704924, 87679850, 126697122, 126727561, 126941873]

Max de RT :  99 / max de rép : 81 / max de likes :  389

Moyennes de RT :  0.5299050437785177 / de rép : 0.20113454186706128 / de likes :  1.2114934023924036

Hashtags :  [['reformedesretraites', 945]

In [18]:
"""À faire qu'une fois (résolu ensuite dans la requête directement)"""
#supprimer les tweets pas en français (récupérés avant l'ajout de la contrainte lang:fr), ne concernait qu'une soixantaine de tweets/1200

for doc in glob.glob("corpus_projet_plus/*"):
    corpus = ouvrir_json(doc)
    #dico structure : 3 grandes clés : data, includes (media), meta
    data = corpus["data"]
    len_data = len(data)
    includes = corpus["includes"]
    meta = corpus["meta"]
        
    """PARTIE DATA"""

    for tweet in data:
        """TEXT"""
        text = tweet["text"].lower()

        """LANG"""
        lg[tweet['lang']] = lg.get(tweet['lang'], 0) + 1
        if tweet['lang']!='fr':
            print(tweet['lang'],text)
            print("********")
            data.remove(tweet)
            
    if len_data != len(data):
        with open(doc, 'w') as f:
            json.dump(corpus, f, indent=4)

zxx https://t.co/30yzrhspgv
********
und 🤗
#greve19janvier 
#reformedesretraites 
#retraite https://t.co/r3t41kj905
********
en @brunolemaire macron #reformedesretraites #greve19janvier #lycee 

#grevegeneraleillimitee 
#grevegenerale19janvier 

https://t.co/rd3gq5aid2
********
qme #montedemarsan #greve19janvier #reformedesretraites #collterr #secretairedemairie https://t.co/x1x383uebv
********
en prue
jay briscoe
#brandnamesinfilmsorsongs
premier league
depay
josh norris
truth social
memphis
homepod
#milaninter
ghostface
#pistorius
narty
walker howard
#réformedesretraites
ausgabe
kampagne
melonik
nick khan https://t.co/tkzkfmlrwo
********
qme @alexishrzs #le19janvierjetravaille 
#reformedesretraites https://t.co/wldwne8fue
********
